# Python Script for Importing from a CSV File into a PostgreSQL database

This script takes https://youtu.be/wqBFgaMgFQA as an example/inspiration.

The main motivation for using this script is due to SQL being very useful for finding insights in a database. At the time of writing, I'm not as proficient with Python as compared to SQL.

Thus, being able to push a csv database into a PostgreSQL database is much more useful for manipulation/queries later as a next step.

#### Steps

- Import from csv into a pandas df
- Clean the table name and remove all extra symbols, spaces, capital letters
- Clean the column headers and remove all extra symbols, spaces, capital letters
- Write the create table SQL statement
- Import the data into the database

In [3]:
# imports the necessary libraries
# os is for allowing python to access and modify directories

import os
import numpy as np
import pandas as pd

# psycopg2 is a postgresql adapter for python
import psycopg2


In [4]:
# look for all files in this directory
# this is a bash/shell command; not a python code

!ls

Untitled.ipynb
python_to_sql_upload.ipynb
total_pop_state_more_than_3rd_least_dense_region.sql


In [5]:
df = pd.read_csv("homelessness.csv")
df.head()

,region,state,individuals,family_members,state_pop
0,East South Central,Alabama,2570.0,864.0,4887681
1,Pacific,Alaska,1434.0,582.0,735139
2,Mountain,Arizona,7259.0,2606.0,7158024
3,West South Central,Arkansas,2280.0,432.0,3009733
4,Pacific,California,109008.0,20964.0,39461588


In [6]:
#clean table names
# only be lower case letters
# remove all white spaces and $
# replace -, /, \\ with _

file = "homelessness"

# note the \ is used to break code into new lines instead of one long line. looks cleaner
    # note no space after the \
# note the r forces python to recognize those symbols as raw string instead of parentheses or slash
clean_table_name = file.lower().replace(" ","_").replace("?", "") \
                    .replace("-", "_").replace(r"/", "_").replace("\\","_").replace("%","") \
                    .replace(")", "").replace(r"(", "").replace("$", "")

clean_table_name

'homelessness'

In [7]:
#clean header names
# only be lower case letters
# remove all white spaces and $
# replace -, /, \\ with _

df.columns = [x.lower().replace(" ","_").replace("?", "") \
                    .replace("-", "_").replace(r"/", "_").replace("\\","_").replace("%","") \
                    .replace(")", "").replace(r"(", "").replace("$", "")for x in df.columns]

df.columns

Index(['region', 'state', 'individuals', 'family_members', 'state_pop'], dtype='object')

In [9]:
df.dtypes

region             object
state              object
individuals       float64
family_members    float64
state_pop           int64
dtype: object

In [10]:
# creating a SQL statement to create the database table
# the column types should match the columns imported into pandas, but are the sql equivalents
# it's fine if this gives an error; it's SQL code and not python

create table homelessness
(

    region         varchar,
    state          varchar,
    individuals    float,
    family_members float,
    state_pop      int

);

SyntaxError: invalid syntax (870449372.py, line 5)

In [11]:
# a pandas dictionary for holding the equivalent python column type and sql column type pairings
# we will use a for loop later to automatically do the conversion to pass into the would be database

replacements = {
   
    "object": "varchar",
    "float64": "float",
    "int64": "int",
    "datetime64": "timestamp",
    "timedelta64[ns]": "varchar"
    
}

replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'int',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [13]:
# for loop for converting from python column type to sql column type

column_string = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
column_string

'region varchar, state varchar, individuals float, family_members float, state_pop int'

In [15]:
# open a database connection
# database is a postresql database - that's why we needed to import psycopg2 in the beginning

db_name = "dbname"
db_user = "dbusername"
db_password = "dbpassword"
db_host = "blahblahblah.rds.amazonaws.com"
db_port = "portnum"

conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
)

if conn is not None:
    print('opened database successfully')

    cursor = conn.cursor()

# if nothing is printing, check the connection - especially security group settings
# by default the default security group blocks all ip addressess - add whitelist, or for simplicity allow all ip addressess

OperationalError: invalid integer value "portnum" for connection option "port"


In [16]:
# drop tables with the same name

cursor.execute("drop table if exists homelessness;")

NameError: name 'cursor' is not defined

In [17]:
# create table
# copy the output from [10] cuz that's the cleaned output we want to use for the sql table

cursor.execute("create table homelessness\
(region varchar, state varchar, individuals float, family_members float, state_pop int)")


NameError: name 'cursor' is not defined

In [18]:
# insert values into table

# saving df to csv
df.to_csv('homelessness_cleaned.csv', header = df.columns, index = False, encoding = 'utf-8')

# open the csv file, save it as an object
my_file = open('homelessness_cleaned.csv')
print('file opened in memory')

file opened in memory


In [19]:
# upload file to db
# file is open in memory right now; not as a file - so the method to upload is not a simple "INSERT" statement
# we're copying the entire object in memory and all its values

SQL_STATEMENT = """
COPY homelessness FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to db')

NameError: name 'cursor' is not defined

In [20]:
# granting multiple people access to the table if the default setting of the table is to not let other people to connect

cursor.execute("grant select on table homelessness to public")
conn.commit()

# closing the connection
cursor.close()
print('table homelessness imported into db completed')

NameError: name 'cursor' is not defined